## Final Project Submission

Please fill out:
* Student name: Danyal Andriano
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 


In [1]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import stats
from math import sqrt
import sqlite3
import seaborn as sns
import missingno as msno
from pandasql import sqldf
%matplotlib inline

In [2]:
#Import data 
df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


## Data Clean-up

In [3]:
# Column headings stripped of white spaces and made lower case for easier processing
df.columns.str.lower().str.strip()

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

### Correcting Data Type

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
id               21597 non-null int64
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(8), int64(11), object(2)
memory usage: 3.5+ MB


The data consists of 20 variable columns and an index. There are 21596 entries. Since n far exceeds p (the number of variables in the set) we should have no power issues. There appears to be missing data for certain variables. 

In [15]:
df_num = df_num[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'floors', 'view', 'grade', 'sqft_above']]

In [17]:
#df_num.price.value_counts(normalize=True) < 0.01;

In [18]:
df_z = pd.DataFrame()
for col in df_num.columns:
    df_z[col] = (df_num[col] - df_num[col].mean())/df_num[col].std()
df_z.head()


,price,bedrooms,bathrooms,sqft_living,floors,view,grade,sqft_above
0,-0.866696,-0.402894,-1.451039,-0.980629,-0.915531,-0.305429,-0.560787,-0.735234
1,-0.006251,-0.402894,0.174482,0.533357,0.937409,-0.305429,-0.560787,0.460766
2,-0.980751,-1.482459,-1.451039,-1.427201,-0.915531,-0.305429,-1.413157,-1.230546
3,0.173405,0.676671,1.149794,-0.131054,-0.915531,-0.305429,-0.560787,-0.892284
4,-0.082469,-0.402894,-0.150622,-0.436030,-0.915531,-0.305429,0.291583,-0.131194


In [19]:
merged = df_num.merge(df_z, suffixes=('', '_z'), left_index=True, right_index=True)
merged.head()
merged2 = merged
merged2.head()

,price,bedrooms,bathrooms,sqft_living,floors,view,grade,sqft_above,price_z,bedrooms_z,bathrooms_z,sqft_living_z,floors_z,view_z,grade_z,sqft_above_z
0,221900.0,3,1.00,1180,1.0,0.0,7,1180,-0.866696,-0.402894,-1.451039,-0.980629,-0.915531,-0.305429,-0.560787,-0.735234
1,538000.0,3,2.25,2570,2.0,0.0,7,2170,-0.006251,-0.402894,0.174482,0.533357,0.937409,-0.305429,-0.560787,0.460766
2,180000.0,2,1.00,770,1.0,0.0,6,770,-0.980751,-1.482459,-1.451039,-1.427201,-0.915531,-0.305429,-1.413157,-1.230546
3,604000.0,4,3.00,1960,1.0,0.0,7,1050,0.173405,0.676671,1.149794,-0.131054,-0.915531,-0.305429,-0.560787,-0.892284
4,510000.0,3,2.00,1680,1.0,0.0,8,1680,-0.082469,-0.402894,-0.150622,-0.436030,-0.915531,-0.305429,0.291583,-0.131194


In [26]:
len(merged.loc[merged['price_z'] > 2.5])/len(merged)
#merged = (merged.iloc[:,8:] > 3) | (merged.iloc[:,8:] < 3) 

0.026207343612538778

- sqft_basement is float, but contains place holders for unknown data. Explore further below for decision to be made. 
- waterfront appears to be bool, but to have missing values; left for further exploration below
- view appears to be bool but to also have some missing values; left for further analysis below
- year data is interval; further analysis to determine how feature will be used for modeling
- zipcode and location data will be removed from the main dataset for now and returned to later... there are too many cities and zipcodes to use as a categorical set of predictors in the model. However, location data could be used to access other demographic data and zipcodes can be used grossly to divide the county into 4 segments (East, West, North, South) if time permits. 

### Drop Columns

In [ ]:
location_df = df.iloc[:, 14:17]
location_df.head()

In [ ]:
# remove date - not time series analysis and we have the years as time reference better suited to an OLS regression. 
# remove id as the integer index appears to be a simpler and more effective identification
# remove location data for now
to_drop = ['date', 'zipcode', 'lat', 'long']
df.drop(to_drop, axis=1, inplace=True)


### Placeholders and Missing Data

In [ ]:
# check for obvious placeholders 

def placeholder(data, placeholder, missing = np.NaN):
    """
    DocString for placeholder function:
    ___________________________________
    input
    data: dataframe, iterable
    placeholder: int, str, float
    placeholder2: np.NaN
    
    return: dict of counts
    """
    counts = 0
    counts2 = 0
    placeholder_lst = []
    for col in data.columns:
        for value in data[col]:
            if value == placeholder:
                counts += 1
            elif value == missing:
                counts2 += 1
    placeholder_lst.append({'Column': col, placeholder: counts, missing: counts2})
    return placeholder_lst

In [ ]:
placeholder(df, '?')

In [ ]:
# df.sqft_basement = df.sqft_basement.astype(float)

In [ ]:
msno.matrix(df);

Notable missing data were apparent for three variables - waterfront, view and yr_renovated. These variables will be further explored to determine how best to deal with the missing values.

In [ ]:
df.isna().sum()

In [ ]:
# Waterfront indicates whether a house has a view to the waterfront. 
df.waterfront.unique()
df.waterfront.value_counts()
 

<img src="king_county.png" style="width: 700px">

Further inspection shows that this data type is in fact boolean. It is not indicated whether a value of 1 means that a view is present, 
however, inspecting the county visually suggetss that a minority of area borders water. We can assume based on this that only 
146 houses in this dataset are on the waterfront.

In [ ]:
#Null values were temporarily dropped to explore further
df.waterfront.dropna(axis=0, inplace=True);
df.waterfront.isna().sum()


In [ ]:
waterfront_ave = round(df.groupby(['waterfront']).mean(), 2)
waterfront_ave.head()

In [ ]:
# replace waterfront missing values with NaN category

df.waterfront = df.waterfront.astype(str)

df.waterfront.replace(np.NaN, 'NaN', inplace=True)
df.waterfront.replace(0.0, 'False', inplace=True)
df.waterfront.replace(1.0, 'True', inplace=True)

In [ ]:
f, axes = plt.subplots(1,2, figsize=(12,5))
sns.boxplot(x="waterfront", y="price", data=df, ax=axes[0])
sns.boxplot(x="waterfront", y="sqft_living", data=df, ax=axes[1])
f.tight_layout()

In [ ]:
# waterfront contains only a very small percentage of houses; however, there appears to be a real difference in size and price. 
# These houses are perhaps best left out of the analysis altogether, as they may represent outliers and a different population. 

### EXTRA

In [ ]:
#### calculate outliers using z-scores and standard deviations
def extreme_values(data):
    z_dict = []
    for col in data.columns:
        if(data[col].dtype == float or data[col].dtype == int):
            for value in data[col]:
                id_val = data.loc[data['id'] == value]
                z = (value - data[col].mean())/data[col].std()
                if z > 4 or z < -4:
                    z_dict.append({'id': id_val, 'variable': col, 'extreme_value': value, 'z_score': z})
    extreme = pd.DataFrame(z_dict)
    return extreme

In [ ]:
threshold = 0.03
for col in df_num2:
    counts = df_num2[col].value_counts(normalize=True)
    df_num3 = df_num2.loc[df_num2[col].isin(counts[counts > threshold].index), :]

In [ ]:
#df_num[(np.abs(stats.zscore(df)) < 3).all(axis=1)]